In [1]:
import nest_asyncio
import asyncio
from ib_insync import *
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import time

nest_asyncio.apply()

# Connect to IBKR TWS or Gateway
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=2)

def Now():
    """Return the current time as a string."""
    return time.strftime('%Y-%m-%d %H:%M:%S')

def fetch_historical_data(contract, duration='5 D', bar_size='1 min'):
    """Fetch historical data for a given contract."""
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr=duration,
        barSizeSetting=bar_size,
        whatToShow='MIDPOINT',
        useRTH=True
    )
    if bars:
        df = util.df(bars)
        print("%s Historical data fetched: %s" % (Now(), df.head()))
        return df
    else:
        print("%s No historical data fetched" % Now())
        return None

def estimate_ou_parameters(df):
    """Estimate parameters of the Ornstein-Uhlenbeck process."""
    dt = 1 / (24 * 60)  # 1-minute data
    x = df['close'].values
    n = len(x)

    # Estimate parameters
    def neg_log_likelihood(params):
        mu, theta, sigma = params
        if sigma <= 0 or mu <= 0:
            return np.inf
        likelihoods = -np.log(sigma) - 0.5 * (((x[1:] - x[:-1] * np.exp(-mu*dt) - theta * (1 - np.exp(-mu*dt))) / sigma) ** 2)
        return -np.sum(likelihoods)

    initial_params = [0.1, np.mean(x), np.std(x)]
    bounds = ((1e-5, None), (None, None), (1e-5, None))  # Avoid zero or negative values
    result = minimize(neg_log_likelihood, initial_params, bounds=bounds)
    
    if not result.success:
        raise ValueError("Parameter estimation failed")

    mu, theta, sigma = result.x
    print("%s Estimated parameters - mu: %s, theta: %s, sigma: %s" % (Now(), mu, theta, sigma))
    return mu, theta, sigma

def calculate_entry_exit_levels(mu, theta, sigma, r, c, stop_loss):
    """Calculate optimal entry and exit levels based on OU parameters and constraints."""
    def value_function_exit(x):
        return (x - c) * np.exp(-r * (1 / mu) * np.log(x / theta))

    def value_function_entry(x):
        return value_function_exit(x) - x - c

    b_star = minimize(lambda x: -value_function_exit(x), x0=theta, bounds=((stop_loss, None),)).x[0]
    a_star = minimize(lambda x: -value_function_entry(x), x0=theta, bounds=((stop_loss, b_star),)).x[0]
    print("%s Calculated levels - a_star: %s, b_star: %s" % (Now(), a_star, b_star))
    return a_star, b_star

def place_market_order(contract, action, quantity):
    """Place a market order for a specified quantity."""
    order = MarketOrder(action, quantity)
    trade = ib.placeOrder(contract, order)
    print(f"{Now()} Placing {action} market order for {quantity} units")

    while not trade.isDone():
        ib.waitOnUpdate()

    price = trade.fills[0].execution.avgPrice if trade.fills else None
    if price:
        print(f"{Now()} Placed {action} market order for {quantity} units at price {price}")
    else:
        print(f"{Now()} {action} market order for {quantity} units was not filled")
    return price

def get_account_balance():
    """Retrieve account balance."""
    account_values = ib.accountValues()
    net_liquidation = next((x for x in account_values if x.tag == 'NetLiquidation'), None)
    if net_liquidation:
        return float(net_liquidation.value)
    return 0.0

async def main():
    # Parameters
    check_window = "5 D"
    bar_size = "1 min"
    contract = Stock('AAPL', 'SMART', 'USD')
    r = 0.01  # Example discount rate
    c = 0.01  # Example transaction cost
    stop_loss_percent = 0.01  # Example stop-loss level as a percentage
    trade_amount = 200  # Number of AAPL shares to trade
    hold_seconds = 300  # Hold for 5 minutes (300 seconds)
    wait = 60  # Check every 60 seconds

    # Fetch historical data and estimate parameters
    df = fetch_historical_data(contract, duration=check_window, bar_size=bar_size)
    if df is not None:
        mu, theta, sigma = estimate_ou_parameters(df)
        a_star, b_star = calculate_entry_exit_levels(mu, theta, sigma, r, c, stop_loss_percent)
        initial_balance = get_account_balance()  # Get the initial account balance
        print("%s Initial account balance: $%s" % (Now(), initial_balance))
    else:
        print("%s Historical data could not be fetched. Exiting." % Now())
        return

    while True:
        # Data
        bars = await ib.reqHistoricalDataAsync(contract, endDateTime='', durationStr=check_window, barSizeSetting=bar_size, whatToShow='MIDPOINT', useRTH=True)
        df = util.df(bars)
        print("%s Data downloaded" % Now())

        # Check condition for mean-reversion strategy
        p_movement = round(df["close"].iloc[-1] / df["open"].iloc[0] - 1, 6)
        condition = p_movement > 0.0001 / 100  # 0.01%

        if condition:
            print(f"{Now()} Condition met ({p_movement}), open trade with market order")
            ib.qualifyContracts(contract)
            price_sell = place_market_order(contract, "SELL", trade_amount)
            if price_sell:
                print("%s Trade opened: Sold at %s" % (Now(), price_sell))

                # Hold the trade
                print("%s Holding the trade" % Now())
                await asyncio.sleep(hold_seconds)

                # Close the trade
                print(f"{Now()} Closing the trade with market order")
                price_buy_back = place_market_order(contract, "BUY", trade_amount)
                if price_buy_back:
                    print("%s Trade closed: Bought back at %s" % (Now(), price_buy_back))

                    profit = "%s %%" % round(100 * (price_sell / price_buy_back - 1), 6)
                    print("%s Gross profit from trade: %s" % (Now(), profit))
                else:
                    print("%s Failed to buy back the shares" % Now())
            else:
                print("%s Failed to sell the shares" % Now())
        else:
            print("%s Condition not met (%s), do not open any trade" % (Now(), p_movement))

        # Wait for the next iteration
        print("%s ........ wait for %s secs ........." % (Now(), wait))
        await asyncio.sleep(wait)

# Run the main function using asyncio
asyncio.run(main())


2024-06-07 18:03:13 Historical data fetched:                        date    open    high     low   close  volume  average  \
0 2024-06-03 09:30:00-04:00  192.99  193.34  192.77  193.33    -1.0     -1.0   
1 2024-06-03 09:31:00-04:00  193.33  193.39  192.58  192.78    -1.0     -1.0   
2 2024-06-03 09:32:00-04:00  192.78  192.91  192.52  192.78    -1.0     -1.0   
3 2024-06-03 09:33:00-04:00  192.78  192.98  192.65  192.80    -1.0     -1.0   
4 2024-06-03 09:34:00-04:00  192.80  193.24  192.75  193.14    -1.0     -1.0   

   barCount  
0        -1  
1        -1  
2        -1  
3        -1  
4        -1  
2024-06-07 18:03:13 Estimated parameters - mu: 8.804150692144246, theta: 195.14673748207107, sigma: 0.10279138726017338
2024-06-07 18:03:13 Calculated levels - a_star: 71.7239754716066, b_star: 6.056808599844238e+37
2024-06-07 18:03:13 Initial account balance: $1005580.8
2024-06-07 18:03:13 Data downloaded
2024-06-07 18:03:13 Condition met (0.014768), open trade with market order
2024-06